In [1]:
import numpy as np
import pandas as pd
import json
import pickle
import math

from qiskit.visualization import plot_histogram 
pd.options.display.float_format = '{:,.15f}'.format

In [12]:
# ex. # of partition, partition qubit ([[0,1], [2,3], [7,8], [9,10]]), condition qubit ([4,5,6])
def my_marginal(data, p_q, c_q, qubits, device_type, wh):

    total_partition = len(p_q)
    reduced_q = len(p_q[0])
    partition_q = len(c_q)
    
    ret_p = []
    ret_c = []
    ret_cp = [[] for i in range(partition_q) ]
    for i in data.iterrows():

        dic_p = [{} for i in range(total_partition)]
        for l in range(total_partition):
            for j in range(2 ** partition_q ):
                bit = str(bin(j)[2:]).zfill(partition_q)
                dic_p[l][bit] = {}
                for k in range(2 ** len(p_q[l])):
                    bit_2 = str(bin(k)[2:]).zfill(len(p_q[l]))
                    dic_p[l][bit][bit_2] = 0
        
        dic_c = {}
        for j in range(2 ** partition_q):
            bit = str(bin(j)[2:]).zfill(partition_q)
            dic_c[bit] = 0
        
        dic_cp = [{} for i in range(partition_q)]
        for j in range(partition_q):
            for k in range(2):
                bit = str(bin(k)[2:]).zfill(1)
                dic_cp[j][bit] = 0
                   
        for j, k in zip(i[1], range(len(i[1]))):
            bit = str(bin(k)[2:]).zfill(qubits)   
            selected_chars_c = [bit[qubits - 1 - m] for m in c_q]
            new_string_c = ''.join(selected_chars_c)
            
            for l in range(total_partition):

                selected_chars = [bit[qubits - 1 - m] for m in p_q[l]]
                new_string = ''.join(selected_chars[::-1])
                dic_p[l][new_string_c][new_string] += j
                
            for l in range(partition_q):
                dic_cp[l][bit[qubits - 1 - c_q[l]]] += j

            dic_c[new_string_c] += j            
            
        ret_p.append(dic_p)
        ret_c.append(dic_c)
        for j in range(partition_q):
            ret_cp[j].append(dic_cp[j])
        
    a = [[] for i in range(total_partition * 2 ** partition_q) ]
    for i in ret_p:
        for j in range(len(i)):
            for k in range(len(i[j])):
                bit = str(bin(k)[2:]).zfill(partition_q)
                a[k + j * (2 ** partition_q)].append(i[j][bit])
                
    res_c_df = pd.DataFrame(ret_c)
    res_c_df.to_csv('./' + device_type + '/' + device_type + '_' + str(qubits) +  'qubits' + wh + 'D' + '.csv', index=False, header=True)
    
    for l in range(partition_q):
        res_cp_df = pd.DataFrame(ret_cp[l])
        res_cp_df.to_csv('./' + device_type + '/' + device_type + '_' + str(qubits) +  'qubits' + wh + 'D' + str(l) + '.csv', index=False, header=True)
        
    
    for l in range(len(a)):
        test_df = pd.DataFrame(a[l])
        for i, j, k in zip(range(len(test_df)), test_df.iterrows(), res_c_df.iterrows()):
            test_df.iloc[i] = j[1] / k[1].values[l % (2 ** partition_q)]
        test_df.to_csv('./' + device_type + '/' + device_type + '_' + str(qubits) +  'qubits' + wh + str(l) + '.csv', index=False, header=True)

    #return ret_p, ret_c, a 
    

In [8]:
#n_qubits = [7]
#n_data = 300
#times = [39] # 11700, 39
#device_type = 'ibm_lagos' 
#number = 0

#n_qubits = [7]
#n_data = 300
#times = [34] # 10200, 34
#device_type = 'ibmq_jakarta' 
#number = 0

n_qubits = [13]
n_data = 300 
times = [20]
device_type = 'ibmq_mumbai'
number = 0


In [8]:
X = pd.read_csv('./' + device_type + '/' + device_type + '_' + str(n_qubits[0]) +  'qubits_obs_' + str(number) + '.csv')
Y = pd.read_csv('./' + device_type + '/' + device_type + '_' + str(n_qubits[0]) +  'qubits_ideal_' + str(number) + '.csv')
LI = pd.read_csv('./' + device_type + '/' + device_type + '_' + str(n_qubits[0]) +  'qubits_miti_' + str(number) + '.csv')

In [9]:
X = pd.read_csv('./' + device_type + '/' + device_type + '_' + str(n_qubits[0]) +  'qubits_obs_' + str(number) + '.csv')
Y = pd.read_csv('./' + device_type + '/' + device_type + '_' + str(n_qubits[0]) +  'qubits_ideal_' + str(number) + '.csv')


In [28]:
'''
number = 7

X2 = pd.read_csv('./' + device_type + '/' + device_type + '_' + str(n_qubits[0]) +  'qubits_obs_' + str(number) + '.csv')
Y2 = pd.read_csv('./' + device_type + '/' + device_type + '_' + str(n_qubits[0]) +  'qubits_ideal_' + str(number) + '.csv')
X_m = pd.concat([X, X2], ignore_index = True)
Y_m = pd.concat([Y, Y2], ignore_index = True)
X_m.to_csv(device_type + '_' + str(n_qubits[0]) +  'qubits_obs_' + str(99) + '.csv', index=False, header=True)
Y_m.to_csv(device_type + '_' + str(n_qubits[0]) +  'qubits_ideal_' + str(99) + '.csv', index=False, header=True)
'''

In [13]:
# data, partition qubit, condition qubit, # of qubit, device, name
my_marginal(X, [[0, 2, 3], [8, 11], [1, 4, 5], [10, 12]], [6, 7, 9], 13, device_type, '_obs_CI_')
my_marginal(Y, [[0, 2, 3], [8, 11], [1, 4, 5], [10, 12]], [6, 7, 9], 13, device_type, '_ideal_CI_') 

In [9]:
col = []
for i in range(2**n_qubits[0]):
    bit_i = str(bin(i)[2:]).zfill(n_qubits[0])
    col.append(bit_i)
    
X.columns = col 
Y.columns = col
LI.columns = col

In [ ]:
# data, partition qubit, condition qubit, # of qubit, device, name
my_marginal(X, [[0, 1, 2], [4, 5, 6]], [3], 7, device_type, '_obs_CI_')
my_marginal(Y, [[0, 1, 2], [4, 5, 6]], [3], 7, device_type, '_ideal_CI_') 